# Using a Large Language Model locally

To interact with LLMs I use Ollama https://ollama.com, which can then be used in the terminal or through Python.

I obtained the 7B-parameter version of DeepSeek-R1, with the command:

    ollama pull deepseek-r1:7b

Then I can chat with the model with:

    ollama run deepseek-r1:7b

Now let's do it in Python!

In [2]:
import ollama
LANGUAGE_MODEL = 'deepseek-r1:7b'

## Asking a question with ollama.generate()

In [17]:
%%time
stream = ollama.generate(model=LANGUAGE_MODEL, prompt='How far away is the Sun?', stream=True)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

The distance from Earth to the Sun is approximately 149.6 million kilometers (about 93 million miles). This distance is also known as one astronomical unit (AU), which is a common measure of scale in astronomy.CPU times: user 79.5 ms, sys: 25.2 ms, total: 105 ms
Wall time: 3.52 s


Didn't need to think at all, as it probably had the answere memorised. 

## Asking the same question again

In [18]:
%%time
stream = ollama.generate(model=LANGUAGE_MODEL, prompt='How far away is the Sun?', stream=True)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

The distance from Earth to the Sun is approximately 93 million miles (150 million kilometers).CPU times: user 41.5 ms, sys: 12.7 ms, total: 54.2 ms
Wall time: 1.68 s


Remark that the answer is not exactly the same! The LLM provides a probability for each next word in the text, and Ollama picks one based on those probabilities, which makes every run different.

## How to always get the same output

If we want the output to always be the same, we can force it to always pick the most probable by setting the `temperature` parameter to zero:

In [21]:
%%time
stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt='Is Japan in Asia?', 
    stream=True,
    options={"temperature": 0},
)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

Yes, Japan is located in Asia. It is an island country off the eastern coast of the Pacific Ocean and is part of East Asia.CPU times: user 52.1 ms, sys: 16.3 ms, total: 68.5 ms
Wall time: 2.31 s


In [22]:
%%time
stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt='Is Japan in Asia?', 
    stream=True,
    options={"temperature": 0},
)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

Yes, Japan is located in Asia. It is an island country off the eastern coast of the Pacific Ocean and is part of East Asia.CPU times: user 53.3 ms, sys: 16.2 ms, total: 69.5 ms
Wall time: 2.05 s


Just because the model thinks it knows the answer doesn't mean it is correct! Example:

In [30]:
%%time
stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt='Who are the Rolling Stones?', 
    stream=True,
    options={"temperature": 0},
)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

The Rolling Stones are one of the most iconic and influential rock bands in music history. Formed in 1962 in London, England, by Mick Jagger and Keith Richards, the band has evolved over the years with the addition of other members, including Charlie Chang (bass) and Bill Wyman (guitar). The Rolling Stones are known for their powerful stage presence, energetic performances, and a wide range of music that spans more than five decades.

The band's music is characterized by its deep grooves, emotional lyrics, and a blend of rock, blues, and psychedelic elements. Some of their most famous songs include "Brown Sugar," "Paint It, Black," "Satisfaction," "Hey Jude," "Paint the Sky," and "Tattooed Love." The Rolling Stones have won numerous awards, including five Grammys, and have sold over 100 million records worldwide.

In addition to their music, the Rolling Stones are celebrated for their live performances, which have captivated audiences around the world. Their discogra

## Watching the model think

If we ask simple questions for which the LLM has memorised the answer, it returns an output very quickly. For more complicated requests it will show us its thinking process:

In [33]:
%%time
stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt='What are the legal implications of owning a lawnmower in Iceland?', 
    stream=True,
    options={"temperature": 0},
)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>
Okay, so I need to figure out what the legal implications are for owning a lawnmower in Iceland. Hmm, where do I start? Well, first off, I know that in many places, having a power mower or lawnmower falls under some kind of regulation because it's considered a dangerous appliance. But I'm not exactly sure how Iceland treats this.

I think the first thing to consider is whether owning a lawnmower requires any specific license or permit. In my country, for example, you need a driver's license and maybe a motorcycle license if you want to drive one. So maybe in Iceland, they have something similar. Maybe there are age restrictions too? I mean, can anyone get a lawnmower license, or is it only for people over 18?

Then there's the issue of public safety. Lawnmowers can be dangerous because they're heavy and have sharp blades. So perhaps owning one means you have to register it with local authorities so they know where it is in case someone gets hurt. I remember seeing signs around 

## Providing chat history with ollama.chat()

This module allows you to provide a whole list of messages to the LLM, specifying if they come from the user, or the LLM itself (`assistant` role). This is how you keep a chat history, so you can ask for instance:

    user: What is the capital of Japan?
    assistant: Tokyo.
    user: And Thailand?    

In [60]:
messages = [
  {'role': 'user', 'content': 'What is the capital of Japan?'},
]

response = ollama.chat(LANGUAGE_MODEL, 
                       messages=messages,
                      options={"temperature": 0})
message = response['message']
print(message['content'])
messages.append(message)

<think>

</think>

The capital of Japan is Tokyo.


In [61]:
messages.append({'role': 'user', 'content': 'And Thailand?'})
response = ollama.chat(LANGUAGE_MODEL, 
                       messages=messages,
                      options={"temperature": 0})
message = response['message']
print(message['content'])
messages.append(message)

<think>
Okay, so the user asked about the capitals of Japan and Thailand after I told them that Japan's capital is Tokyo. Now they're asking about Thailand.

I should provide a clear answer first: Bangkok is Thailand's capital.

But wait, maybe they want more details. They might be curious about why Bangkok is the capital or if there are any other capitals besides it.

I should mention that while Bangkok is the official capital, sometimes people refer to Chiang Mai as well, especially in certain contexts like tourism.

Also, I can add a bit about the significance of the capital cities—how they're often the administrative centers and have major economic activities.

Keeping it friendly and informative would be best. Maybe offer further help if they need more details.
</think>

The capital of Thailand is Bangkok.
